In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%run ../preprocessing.py
pd.set_option("display.max_columns", 8)

# 1. Processing cycles information
- Each cycle for each woman needs to be expanded such that there is a row per calendar day of the cycle and with a boolean indicator for period days, the cycle_id and the day in cycle. This is done with the help of `expand_cycle` function, e.g.:

In [3]:
cycles.head(1)

,user_id,cycle_id,cycle_start,cycle_length,period_length,expected_cycle_length
0,03009100-a1fa-4fad-bf9b-6102c690f3be,1,2016-11-27,24.0,1.0,29.0


In [4]:
expand_cycle(cycles.iloc[0])

cycle_id  day_in_cycle  \
user_id                              date                                 
03009100-a1fa-4fad-bf9b-6102c690f3be 2016-11-27         1             1   
                                     2016-11-28         1             2   
                                     2016-11-29         1             3   
                                     2016-11-30         1             4   
                                     2016-12-01         1             5   
                                     2016-12-02         1             6   
                                     2016-12-03         1             7   
                                     2016-12-04         1             8   
                                     2016-12-05         1             9   
                                     2016-12-06         1            10   
                                     2016-12-07         1            11   
                                     2016-12-08         1            12   
                                     2016-12-09         1            13   
                                     2016-12-10         1            14   
                                     2016-12-11         1            15   
                                     2016-12-12         1            16   
                                     2016-12-13         1            17   
                                     2016-12-14         1            18   
                                     2016-12-15         1            19   
                                     2016-12-16         1            20   
                                     2016-12-17         1            21   
                                     2016-12-18         1            22   
                                     2016-12-19         1            23   
                                     2016-12-20         1            24   

                                                 period  
user_id                              date                
03009100-a1fa-4fad-bf9b-6102c690f3be 2016-11-27       1  
                                     2016-11-28       0  
                                     2016-11-29       0  
                                     2016-11-30       0  
                                     2016-12-01       0  
                                     2016-12-02       0  
                                     2016-12-03       0  
                                     2016-12-04       0  
                                     2016-12-05       0  
                                     2016-12-06       0  
                                     2016-12-07       0  
                                     2016-12-08       0  
                                     2016-12-09       0  
                                     2016-12-10       0  
                                     2016-12-11       0  
                                     2016-12-12       0  
                                     2016-12-13       0  
                                     2016-12-14       0  
                                     2016-12-15       0  
                                     2016-12-16       0  
                                     2016-12-17       0  
                                     2016-12-18       0  
                                     2016-12-19       0  
                                     2016-12-20       0

- With the function `expand_cycles` we can iterate over all the cycles:

In [5]:
# Expand cycles so that there is a line per date (active or not) with a boolean indicator of period
cycles_processed = expand_cycles(cycles)

# 2. Processing tracked symptoms
- We need to one hot encode the symptoms and aggregate them such that there is one line per user per active day. Each column will be a boolean indicator of a tracked symptom for a given user and active day. There can be more than one tracked symptom per line.

In [6]:
# Expand tracking so that there is a line per active date with a one hot encoded symtoms.
tracking_processed = process_tracking(tracking)

# 3. Merging extracted features from cycles and tracking
- By doing the merge, information from cycles will bring the boolean indicator of period days and guarantee that every inactive day is also considered. Inactive days are important because RNN need to take into account evolution over time. Intuitively speaking, the RNN "memorizes" how many inactive days are common between tracking activity.
- The first 81 columns refer to symptom, where the first 16 are the symptoms to be predicted
- The last 3 columns refer to cycle information --> cycle_id, day_in_cycle and period

In [7]:
# Merge cycles and tracking information
features = pd.merge(
    tracking_processed,
    cycles_processed,
    left_index=True, right_index=True, how='outer'
).fillna(0)

In [8]:
features

happy  pms  sad  \
user_id                              date                          
00035291-ebfc-4ad3-83da-f83e0cfceeb9 2016-10-05    0.0  0.0  0.0   
                                     2016-10-06    0.0  0.0  0.0   
                                     2016-10-07    0.0  0.0  0.0   
                                     2016-10-08    0.0  0.0  0.0   
                                     2016-10-09    0.0  0.0  0.0   
                                     2016-10-10    0.0  0.0  0.0   
                                     2016-10-11    0.0  0.0  0.0   
                                     2016-10-12    0.0  0.0  0.0   
                                     2016-10-13    0.0  0.0  0.0   
                                     2016-10-14    0.0  0.0  0.0   
                                     2016-10-15    0.0  0.0  0.0   
                                     2016-10-16    0.0  0.0  0.0   
                                     2016-10-17    0.0  0.0  0.0   
                                     2016-10-18    0.0  0.0  0.0   
                                     2016-10-19    0.0  0.0  0.0   
                                     2016-10-20    0.0  0.0  0.0   
                                     2016-10-21    0.0  0.0  0.0   
                                     2016-10-22    0.0  0.0  0.0   
                                     2016-10-23    0.0  0.0  0.0   
                                     2016-10-24    0.0  0.0  0.0   
                                     2016-10-25    0.0  0.0  0.0   
                                     2016-10-26    1.0  0.0  0.0   
                                     2016-10-27    0.0  0.0  0.0   
                                     2016-10-28    0.0  0.0  0.0   
                                     2016-10-29    1.0  0.0  0.0   
                                     2016-10-30    1.0  0.0  0.0   
                                     2016-10-31    1.0  0.0  0.0   
                                     2016-11-01    0.0  0.0  0.0   
                                     2016-11-02    0.0  0.0  0.0   
                                     2016-11-03    0.0  0.0  0.0   
...                                                ...  ...  ...   
fffe9159-a2ca-4b7f-a692-00e43ece3154 2016-09-02    0.0  0.0  0.0   
                                     2016-09-03    1.0  0.0  0.0   
                                     2016-09-04    0.0  0.0  0.0   
                                     2016-09-05    1.0  0.0  0.0   
                                     2016-09-06    0.0  0.0  1.0   
                                     2016-09-07    1.0  0.0  0.0   
                                     2016-09-08    0.0  0.0  0.0   
                                     2016-09-09    0.0  0.0  0.0   
                                     2016-09-10    0.0  0.0  0.0   
                                     2016-09-11    0.0  0.0  0.0   
                                     2016-09-12    0.0  0.0  0.0   
                                     2016-09-13    0.0  0.0  0.0   
                                     2016-09-14    0.0  0.0  0.0   
                                     2016-09-15    0.0  0.0  0.0   
                                     2016-09-16    0.0  0.0  0.0   
                                     2016-09-17    0.0  0.0  0.0   
                                     2016-09-18    0.0  0.0  0.0   
                                     2016-09-19    0.0  0.0  0.0   
                                     2016-09-20    0.0  0.0  0.0   
                                     2016-09-21    0.0  0.0  0.0   
                                     2016-09-22    0.0  0.0  0.0   
                                     2016-09-23    0.0  0.0  0.0   
                                     2016-09-24    0.0  0.0  0.0   
                                     2016-09-25    0.0  0.0  0.0   
                                     2016-09-26    0.0  0.0  0.0   
                                     2016-09-27    0.0  0.0  0.0   
                                     

# 4. Enumerate days for each user
- Tracking behavior evolves with time. Some users might be more active at first and then use the app less. The RNN could take into account for how long a user has been using the app to estimate if she will keep tracking symptoms.
- The first step for getting this information is to find the first day each user starting using the app:

In [9]:
# Find the first day the user started using the app
features = pd.merge(
    features,
    cycles.groupby('user_id')\
          .agg({'cycle_start': {'first_use': 'min'}})\
          .reset_index()\
          .set_index('user_id')['cycle_start'],
    left_index=True,
    right_index=True
)

- Get the absolute day by substracting first_use from date on the index

In [10]:
# Find the absolute day for each row from the day the user started using the app
absolute_day = (features.reset_index().date.dt.date - features.reset_index().first_use.dt.date).dt.days + 1
# Get back the index from features
absolute_day.index = features.index
features['absolute_day'] = absolute_day
# We no longer need 'first_use' column
features = features.drop('first_use', axis=1)

In [11]:
features

happy  pms  sad  \
user_id                              date                          
00035291-ebfc-4ad3-83da-f83e0cfceeb9 2016-10-05    0.0  0.0  0.0   
                                     2016-10-06    0.0  0.0  0.0   
                                     2016-10-07    0.0  0.0  0.0   
                                     2016-10-08    0.0  0.0  0.0   
                                     2016-10-09    0.0  0.0  0.0   
                                     2016-10-10    0.0  0.0  0.0   
                                     2016-10-11    0.0  0.0  0.0   
                                     2016-10-12    0.0  0.0  0.0   
                                     2016-10-13    0.0  0.0  0.0   
                                     2016-10-14    0.0  0.0  0.0   
                                     2016-10-15    0.0  0.0  0.0   
                                     2016-10-16    0.0  0.0  0.0   
                                     2016-10-17    0.0  0.0  0.0   
                                     2016-10-18    0.0  0.0  0.0   
                                     2016-10-19    0.0  0.0  0.0   
                                     2016-10-20    0.0  0.0  0.0   
                                     2016-10-21    0.0  0.0  0.0   
                                     2016-10-22    0.0  0.0  0.0   
                                     2016-10-23    0.0  0.0  0.0   
                                     2016-10-24    0.0  0.0  0.0   
                                     2016-10-25    0.0  0.0  0.0   
                                     2016-10-26    1.0  0.0  0.0   
                                     2016-10-27    0.0  0.0  0.0   
                                     2016-10-28    0.0  0.0  0.0   
                                     2016-10-29    1.0  0.0  0.0   
                                     2016-10-30    1.0  0.0  0.0   
                                     2016-10-31    1.0  0.0  0.0   
                                     2016-11-01    0.0  0.0  0.0   
                                     2016-11-02    0.0  0.0  0.0   
                                     2016-11-03    0.0  0.0  0.0   
...                                                ...  ...  ...   
fffe9159-a2ca-4b7f-a692-00e43ece3154 2016-09-02    0.0  0.0  0.0   
                                     2016-09-03    1.0  0.0  0.0   
                                     2016-09-04    0.0  0.0  0.0   
                                     2016-09-05    1.0  0.0  0.0   
                                     2016-09-06    0.0  0.0  1.0   
                                     2016-09-07    1.0  0.0  0.0   
                                     2016-09-08    0.0  0.0  0.0   
                                     2016-09-09    0.0  0.0  0.0   
                                     2016-09-10    0.0  0.0  0.0   
                                     2016-09-11    0.0  0.0  0.0   
                                     2016-09-12    0.0  0.0  0.0   
                                     2016-09-13    0.0  0.0  0.0   
                                     2016-09-14    0.0  0.0  0.0   
                                     2016-09-15    0.0  0.0  0.0   
                                     2016-09-16    0.0  0.0  0.0   
                                     2016-09-17    0.0  0.0  0.0   
                                     2016-09-18    0.0  0.0  0.0   
                                     2016-09-19    0.0  0.0  0.0   
                                     2016-09-20    0.0  0.0  0.0   
                                     2016-09-21    0.0  0.0  0.0   
                                     2016-09-22    0.0  0.0  0.0   
                                     2016-09-23    0.0  0.0  0.0   
                                     2016-09-24    0.0  0.0  0.0   
                                     2016-09-25    0.0  0.0  0.0   
                                     2016-09-26    0.0  0.0  0.0   
                                     2016-09-27    0.0  0.0  0.0   
                                     